# Trained Model Interface
> "Why simple, when you can use design patterns?"

In [ ]:
# default_exp train_eval_model

In [ ]:
# hide
import blackhc.project.script
from nbdev.showdoc import *

In [ ]:
# exports
from dataclasses import dataclass
from typing import Type

import torch
import torch.nn
import torch.utils.data
from torch import nn

from batchbald_redux.active_learning import RandomFixedLengthSampler
from batchbald_redux.consistent_mc_dropout import get_log_mean_probs
from batchbald_redux.dataset_challenges import (
    RandomLabelsDataset,
    ReplaceTargetsDataset,
)
from batchbald_redux.trained_model import TrainedModel, ModelTrainer

In [ ]:
# exports


class TrainEvalModel:
    def __call__(self, *, training_log, device) -> TrainedModel:
        raise NotImplementedError()


@dataclass
class TrainSelfDistillationEvalModel(TrainEvalModel):
    num_pool_samples: int
    training_dataset: torch.utils.data.Dataset
    eval_dataset: torch.utils.data.Dataset
    validation_loader: torch.utils.data.DataLoader
    training_batch_size: int
    trained_model: TrainedModel
    model_trainer: ModelTrainer
    min_samples_per_epoch: int
    # TODO: remove the default?
    train_augmentations: nn.Module = None

    def __call__(self, *, training_log, device):
        train_eval_dataset = torch.utils.data.ConcatDataset([self.training_dataset, self.eval_dataset])
        train_eval_loader = torch.utils.data.DataLoader(train_eval_dataset, batch_size=512, drop_last=False)

        eval_log_probs_N_C = get_log_mean_probs(
            self.trained_model.get_log_probs_N_K_C(train_eval_loader, num_samples=self.num_pool_samples, device=device)
        )

        eval_self_distillation_dataset = ReplaceTargetsDataset(dataset=train_eval_dataset, targets=eval_log_probs_N_C)

        # TODO: Dataloaders should be part of the model trainer, too!
        train_eval_self_distillation_loader = torch.utils.data.DataLoader(
            eval_self_distillation_dataset,
            batch_size=self.training_batch_size,
            sampler=RandomFixedLengthSampler(eval_self_distillation_dataset, self.min_samples_per_epoch),
            drop_last=True,
        )

        trained_model = self.model_trainer.get_distilled(
            prediction_loader=train_eval_self_distillation_loader,
            train_augmentations=self.train_augmentations,
            validation_loader=self.validation_loader,
            log=training_log
        )

        return trained_model


@dataclass
class TrainRandomLabelEvalModel(TrainEvalModel):
    num_pool_samples: int
    num_training_samples: int
    num_validation_samples: int
    num_patience_epochs: int
    max_epochs: int
    training_dataset: torch.utils.data.Dataset
    eval_dataset: torch.utils.data.Dataset
    validation_loader: torch.utils.data.DataLoader
    training_batch_size: int
    model_trainer: ModelTrainer
    # TODO: remove the default?
    train_augmentations: nn.Module = None

    def __call__(self, *, training_log, device):
        # TODO: support one_hot!
        # TODO: different seed needed!
        train_eval_dataset = torch.utils.data.ConcatDataset(
            [self.training_dataset, RandomLabelsDataset(self.eval_dataset, seed=0)]
        )
        train_eval_loader = torch.utils.data.DataLoader(
            train_eval_dataset, batch_size=self.training_batch_size, drop_last=True, shuffle=True
        )

        trained_model = self.model_trainer.get_distilled(
            prediction_loader=train_eval_loader,
            train_augmentations=self.train_augmentations,
            validation_loader=self.validation_loader,
            log=training_log
        )

        return trained_model


@dataclass
class TrainExplicitEvalModel(TrainEvalModel):
    num_pool_samples: int
    num_training_samples: int
    num_validation_samples: int
    num_patience_epochs: int
    max_epochs: int
    training_dataset: torch.utils.data.Dataset
    eval_dataset: torch.utils.data.Dataset
    validation_loader: torch.utils.data.DataLoader
    training_batch_size: int
    model_trainer: ModelTrainer
    # TODO: remove the default?
    train_augmentations: nn.Module = None

    def __call__(self, *, training_log, device):
        # TODO: support one_hot!
        # TODO: different seed needed!
        train_eval_dataset = torch.utils.data.ConcatDataset([self.training_dataset, self.eval_dataset])
        train_eval_loader = torch.utils.data.DataLoader(
            train_eval_dataset, batch_size=self.training_batch_size, drop_last=True, shuffle=True
        )

        trained_model = self.model_trainer.get_trained(train_loader=train_eval_loader,
                                                       train_augmentations=self.train_augmentations,
                                                       validation_loader=self.validation_loader, log=training_log)

        return trained_model